Por ejemplo considere el caso de querer estimar, usando una variable gaussiana 1D, la distancia al siguiente auto en una ruta contando la cantidad de pixels en su silueta.

# Modelo de contingencia de las variables de estado del mundo $w$ sobre los datos $x$

$$
P(W \mid X)
$$
Dado que el model es univarial y continuo, eligimos la variable guassiana 1D, para ello fijamos la  $\sigma^2$ (varianza) (Porque??) y hacemos que $\mu$ este en funcion de los datos, asi tenemos dos variables nuevas:
$ \phi_0 + \phi_1 $
y la funcion de probabilidad queda como:
$$
Pr(w \mid x, \theta)
= \mathrm{Norm}_w\left[ \phi_0 + \phi_1 x,\, \sigma^2 \right]
$$
Por lo que los parametros del modelo son:
$$
\theta = \{ \phi_0,\ \phi_1,\ \sigma \}
$$
Esta formulacion se conoce como *Linear Regression*
El algoritmo de aprendizaje para este modelo enparda $\{ x_i,\ w_i \}_{i=1}^{I}$ los datos del dataset con los parametros del modelo.
El enfoque **MAP** seria:
$$
\hat{\theta}
= \underset{\theta}{\arg\max}\ \big[ Pr(\theta \mid w_{1\ldots I}, x_{1\ldots I}) \big]
$$

$$
= \underset{\theta}{\arg\max}\ \big[ Pr(w_{1\ldots I} \mid x_{1\ldots I}, \theta)\ Pr(\theta) \big]
\quad\text{(por Bayes)}
$$

$$
= \underset{\theta}{\arg\max}\ 
\left[
\prod_{i=1}^{I} Pr(w_i \mid x_i,\ \theta)\ Pr(\theta)
\right]
$$

# Aclaracion frase del libro

"where we have assumed that the I training pairs {x_i , w_i}_{i=1}^I are independent, and defined a suitable prior Pr(θ)."

A continuación se explica cada parte.

---

## 1. Independencia de los pares (x_i, w_i)

El dataset contiene pares:

(x_1, w_1), (x_2, w_2), ..., (x_I, w_I)

Asumir que son independientes significa que cada observación del dataset no depende de las demás. Esto permite factorizar la probabilidad conjunta de esta forma:

Pr(w_1,...,w_I | x_1,...,x_I, θ) = ∏_{i=1}^I Pr(w_i | x_i, θ)

Esta suposición se llama i.i.d. (independiente e idénticamente distribuido).

---

## 2. Definir un prior Pr(θ)

Los parámetros del modelo son:

θ = {φ_0, φ_1, σ}

En el enfoque bayesiano, estos parámetros tienen distribuciones previas antes de ver los datos. Por ejemplo:

φ_0 ~ N(0, 100)
φ_1 ~ N(0, 100)
σ ~ HalfNormal(5) -> Significa distribución Half-Normal con escala 5.

Este prior Pr(θ) representa nuestras creencias iniciales sobre los parámetros.

---

## 3. Cómo se combinan independencia + prior con Bayes

Usando la regla de Bayes:

Pr(θ | datos) ∝ Pr(datos | θ) * Pr(θ)

Y usando independencia:

Pr(datos | θ) = ∏_{i=1}^I Pr(w_i | x_i, θ)

Esto lleva a la expresión para MAP:

θ^ = argmax_θ [ ∏_{i=1}^I Pr(w_i | x_i, θ) * Pr(θ) ]

---

## Resumen

- Asumimos que cada ejemplo del dataset es independiente.
- Definimos un prior sobre los parámetros para hacer inferencia bayesiana.
- Al combinar prior y likelihood (producto por independencia), obtenemos la ecuación usada para estimación MAP.






# A continuacion se va a realizar un ejemplo con la base de datos de Kitti
1.- data_object_image_2.zip -> es la base de datos principal
2.- data_object_label_2.zip
3.- data_object_calib.zip
# Estrategia general para el ejercicio
1.- Detectar si la imagen contien o no contiene un auto
2.- Si contiene extraer la region de la imagen donde  esta el auto
3.- Mediante un proceso de background removal extraeer el contorno del auto
    - Vamos a usar una opcion mas realista y dura para obtener el contorno del auto una vez detectado, mediante la segmentación dentro del bbox (GrabCut / MaskRCNN / SAM, etc.)
    - Hay que validar el proceso anterior (Otra VA que diga si el contorno es valido o no)
    - Despues contamos pixeles del resultado
4.- Estimar la distancia del vehiculo usando Regression (El algoritmo de inferencia del libro)
5.- Que distancia?
    Distancia desde el centro óptico de la cámara al centro 3D del auto
    Porque eso es exactamente lo que representa z en KITTI Object Detection. Es la profundidad respecto a la cámara izquierda.
# Formato del archivo label de KITTI
```text
0:type
1:truncated
2:occluded
3:alpha
4:left
5:top
6:right
7:bottom
8:height
9:width
10:length
11:x
12:y
13:z     <--- distancia en metros
14:rotation_y
```

In [ ]:
import tensorflow as tf
from pathlib import Path

def kitti_pairs_tf_datasets(
    root_abs_path,
    test_size=0.2,
    val_size=0.1,
    seed=42,
    shuffle=True,
):
    """
    Crea tres tf.data.Dataset (train, val, test) con pares (img_path, label_path).

    root_abs_path: str
        Path absoluto donde existen las carpetas image_2/ y label_2/

    test_size: float
        Porcentaje de test (0.2 = 20%)

    val_size: float
        Porcentaje de validation (0.1 = 10%)

    Uso típico en Jupyter:
        train_ds, val_ds, test_ds = kitti_pairs_tf_datasets("/ruta/kitti/training")
    """

    root = Path(root_abs_path).expanduser().resolve()
    img_dir = root / "image_2"
    label_dir = root / "label_2"

    # Verificar existencia
    if not img_dir.exists():
        raise FileNotFoundError(f"No existe {img_dir}")
    if not label_dir.exists():
        raise FileNotFoundError(f"No existe {label_dir}")

    # Listar imágenes
    img_paths = sorted([str(p) for p in img_dir.glob("*.png")])
    if len(img_paths) == 0:
        raise ValueError(f"No se encontraron imágenes PNG en {img_dir}")
    print(f"cantidad de imagenes cargadas: {len(img_paths)}")
    # Emparejar images - labels
    pairs = []
    for img_path in img_paths:
        frame_id = Path(img_path).stem
        label_path = label_dir / f"{frame_id}.txt"
        if label_path.exists():
            pairs.append((img_path, str(label_path)))

    if len(pairs) == 0:
        raise ValueError("No se encontraron pares imagen-label")

    # Convertir a tensores
    img_tensor = tf.constant([p[0] for p in pairs], dtype=tf.string)
    lab_tensor = tf.constant([p[1] for p in pairs], dtype=tf.string)

    n = tf.shape(img_tensor)[0]

    # Shuffle
    if shuffle:
        idx = tf.random.shuffle(tf.range(n), seed=seed)
        img_tensor = tf.gather(img_tensor, idx)
        lab_tensor = tf.gather(lab_tensor, idx)

    # Calcular tamaños
    n_test = int(n.numpy() * test_size)
    n_val  = int(n.numpy() * val_size)
    n_train = n.numpy() - n_test - n_val

    # Particiones
    train_imgs = img_tensor[:n_train]
    train_labs = lab_tensor[:n_train]

    val_imgs = img_tensor[n_train:n_train + n_val]
    val_labs = lab_tensor[n_train:n_train + n_val]

    test_imgs = img_tensor[n_train + n_val:]
    test_labs = lab_tensor[n_train + n_val:]

    # tf.data.Dataset
    train_ds = tf.data.Dataset.from_tensor_slices((train_imgs, train_labs))
    val_ds   = tf.data.Dataset.from_tensor_slices((val_imgs, val_labs))
    test_ds  = tf.data.Dataset.from_tensor_slices((test_imgs, test_labs))

    return train_ds, val_ds, test_ds
